<a href="https://colab.research.google.com/github/andrewscouten/CS-7313/blob/main/Assignment2_knn_classification_yzb2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2 - kNN Classification with Optimizations


## Task 1 - Baseline 1-NN Model & Foundational Concepts
(Train and Evaluate a 1-NN Classifier on the Connect-4 Dataset)

### 1.1 - Implementation: 1-NN Classifier

You can upload the dataset into your Google Drive and load it from there. The code below allows you to mount the Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 1.1.1 - Import the necessary libraries

In [ ]:

# Importing the libraries

# YOUR CODE HERE



#### 1.1.2 - Load and pre-process the dataset

In [ ]:


# Replace this with the correct path on your Google Drive

column_names = ['a1', 'a2', 'a3', 'a4', 'a5', 'a6',
                'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
                'c1', 'c2', 'c3', 'c4', 'c5', 'c6',
                'd1', 'd2', 'd3', 'd4', 'd5', 'd6',
                'e1', 'e2', 'e3', 'e4', 'e5', 'e6',
                'f1', 'f2', 'f3', 'f4', 'f5', 'f6',
                'g1', 'g2', 'g3', 'g4', 'g5', 'g6',
                'class']

# Load the data into a pandas dataframe using the column names defined above
# YOUR CODE HERE


In [ ]:
# Print the shape of the dataframe
# YOUR CODE HERE


# Show the first 5 rows of the dataframe
# YOUR CODE HERE


In [ ]:
# Check for null entries or missing values
# YOUR CODE HERE


#### Convert the class labels to numerical values
Use the `map` function to convert the class labels to numerical values, similar to what we did in the Iris dataset.

In [ ]:
# Converting the class labels to numerical values

label_dict = {
    'win':0,
    'loss':1,
    'draw':2
    }

# YOUR CODE HERE


In [ ]:
# Show a histogram of the class labels. Use the 'class' column for this.

# YOUR CODE HERE


#### Convert the attributes to numerical values

Use the `applymap` function and a `lambda` function to convert the attributes to numerical values.

In [ ]:
# attribute map
attribute_dict = {
    'x':0,
    'o':1,
    'b':2
    }

# YOUR CODE HERE



#### 1.1.3 - Split the dataset into training and testing sets

In [ ]:
# Assign the features to a numpy matrix called X and the class labels to a numpy vector called y

# YOUR CODE HERE


(67557, 42)
(67557,)


In [ ]:
# Split the data into training and test sets. Use 30% of the data for testing and a random state of 42

# YOUR CODE HERE



In [ ]:
# Print the shapes of the training and test sets

# YOUR CODE HERE


(47289, 42)
(20268, 42)


#### 1.1.4 - Train and evaluate a kNN classifier

Train a kNN classifier using the training set and predict the labels of the test set. Use **k=1**. Report the accuracy of the classifier.

**Note:** As a distance metric, use the **Hamming** distance metric. Refer to the documentation of the `sklearn.neighbors.KNeighborsClassifier` class. The metric can be specified using the `metric` parameter of the `KNeighborsClassifier`, as `metric='hamming'`.


In [ ]:
# Fit the KNN model to the training data

# YOUR CODE HERE


KNeighborsClassifier(metric='hamming', n_neighbors=1)

In [ ]:
# Evaluate the model on the test data using classification accuracy and report the results.
# Note, this may take some time to run. It takes about 30 seconds on my machine.

# YOUR CODE HERE



### 1.2 - Write-up: Going Beyond Accuracy

*(Edit the contents of this cell to write your answers according to the assignment description given on Canvas.)*

## Task 2: Hyperparameter Tuning & Model Generalization

### 2.1 - Implementation: Finding the Optimal k

### Split the data into training, validation and test sets, using a 60:20:20 split and follow the instructions given on Canvas.

In [ ]:
# Split the data into training, validation and test sets. Use a 60:20:20 split. Use a random state of 42.

# YOUR CODE HERE



### 2.2 - Write-up: Analysis of Model Complexity and Generalization

*(Edit the contents of this cell to write your answers according to the assignment description given on Canvas.)*


## Task 3: Algorithmic Optimization & Complexity Analysis

The provided naive implementation of kNN is computationally expensive. You will analyze its complexity and then implement an optimized version using a priority queue (max-heap) to see the practical difference.



In [ ]:
# Basic (naive) KNN implementation
from scipy.spatial.distance import hamming

class KNNClassifier(object):
    def __init__(self, k, dist_fn=None):
        self.k = k
        if dist_fn is None:
            self.dist_fn = self._hamming_dist

    def _hamming_dist(self, a, b):
        hamming_distance = hamming(a, b) * len(a)
        return hamming_distance

    def _euclidean_dist(self, a, b):
        dist = 0.
        for ele_i, ele_j in zip(a, b):
            dist += ((ele_i - ele_j) ** 2)
        dist = dist ** 0.5
        return dist

    def _find_nearest(self, x):
        dist_idx_pairs = []
        idx_set = set()
        for k in range(self.k):
            min_dist = self.dist_fn(x, self.dataset_[0])
            min_idx = 0
            for j in range(1, self.dataset_.shape[0]):
                if j in idx_set:
                    continue
                d = self.dist_fn(x, self.dataset_[j])
                if d < min_dist:
                    min_dist = d
                    min_idx = j
            dist_idx_pairs.append((min_dist, min_idx))
            idx_set.add(min_idx)
        return dist_idx_pairs

    def fit(self, X, y):
        self.dataset_ = X.copy()
        self.labels_ = y.copy()
        self.possible_labels_ = np.unique(y)

    def predict(self, X):
        predictions = np.zeros(X.shape[0], dtype=int)
        for i in range(X.shape[0]):
            k_nearest = self._find_nearest(X[i])
            indices = [entry[1] for entry in k_nearest]
            k_labels = self.labels_[indices]
            counts = np.bincount(k_labels,
                                 minlength=self.possible_labels_.shape[0])
            pred_label = np.argmax(counts)
            predictions[i] = pred_label
        return predictions

    def score(self, X, y):
        y_pred = self.predict(X)
        num_correct_predictions = (y_pred == y).sum()
        score = (num_correct_predictions / y.shape[0])
        return score

Python provides `heapq`, which is a MinHeap, i.e. the smallest elements float to the top. The following implementation uses `heapq` internally to create a MaxHeap.You may use it in your implementation of the KNN that uses a priority queue.

In [ ]:
from scipy.spatial.distance import hamming
import heapq


class MaxHeap:

    # Initialize the max heap
    def __init__(self, data=None):
        if data is None:
            self.data = []
        else:
            self.data = [-i for i in data]
            heapq.heapify(self.data)

    # Push item onto max heap, maintaining the heap invariant
    def push(self, item):
        heapq.heappush(self.data, -item)

    # Pop the largest item off the max heap, maintaining the heap invariant
    def pop(self):
        return -heapq.heappop(self.data)

    # Pop and return the current largest value, and add the new item
    def replace(self, item):
        return heapq.heapreplace(self.data, -item)

    # Return the current largest value in the max heap
    def top(self):
        return -self.data[0]

    def size(self):
        return len(self.data)

    def __repr__(self):
        listToStr = ' '.join([str(elem) for elem in self.data])
        return listToStr + '\n'




class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    # Override the less-than operator __lt__ to make Pair class work with max heap
    def __lt__(self, other):
        return self.x < other.x

    def __gt__(self, other):
        return self.x > other.x

    def __neg__(self):
        return Pair(-self.x, self.y)

    def __repr__(self):
        return f'({self.x}, {self.y})'


### 3.1 - Write-up: Theoretical Analysis

*(Edit the contents of this cell to write your answers according to the assignment description given on Canvas.)*


### 3.2 - Implementation: Fast KNN and Empirical Analysis

Implementation of the KNN classifier that is faster than the naive version of the KNN given above, by using a pririty queue.

In [ ]:
# Faster KNN using heap


    # YOUR CODE HERE

## Evaluation

Use a subset of the connect dataset containing only 1500 instances (500 from each class) to compare the **speed** and **accuracy** of the two KNN impelemtations. Use a train/test split of 70/30 and evaluate both version on the same splits with a **k=13**.

In [ ]:
# YOUR CODE HERE
